### Persistence time analysis

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from scipy.stats import norm
from scipy import stats
import json

In [56]:
"""
def filter_data(data_in, max_step_size, window, restriction=10000):
    
    Only keeps data for tracks that are long enough for hurst exponent estimation at a given downsampling step size.
    Args: 
        data_in: pandas dataframe, original data
        max_step_size: int, maximum downsampling step size
        window: int, size of rolling window for hurst component estimation
    Returns:
        filtered_data: pandas dataframe
    
    tracks_to_keep = data_in.TrackID.value_counts().loc[lambda x: (x//max_step_size) > (window+1)].reset_index()['index']
    print(tracks_to_keep)
    if restriction<len(tracks_to_keep):
        filtered_data = data_in[data_in['TrackID'].isin(tracks_to_keep[:restriction])]
    else:
        filtered_data = data_in[data_in['TrackID'].isin(tracks_to_keep)]
    return filtered_data
"""
def filter_data_new(data_in, restriction=10000):
    """
    Only keeps data for tracks that are long enough for hurst exponent estimation at a given downsampling step size.
    Args: 
        data_in: pandas dataframe, original data
        max_step_size: int, maximum downsampling step size
        window: int, size of rolling window for hurst component estimation
    Returns:
        filtered_data: pandas dataframe
    """
    print(data_in)
    tracks_to_keep = data_in.TrackID.value_counts().loc[lambda x: x>2].reset_index()['index']

    if restriction<len(tracks_to_keep):
        filtered_data = data_in[data_in['TrackID'].isin(tracks_to_keep[:restriction])]
    else:
        filtered_data = data_in[data_in['TrackID'].isin(tracks_to_keep)]
        
    print(filtered_data)
    return filtered_data

def downsample(data_input, down_int, start_index=0):
    
    data_out = data_input.iloc[start_index::down_int]
    return data_out



In [57]:
def get_mod(v):
    
    return np.sqrt(v.dot(v))

def get_step_vector(index,data_x,data_y,data_z):
    
    dx = data_x[index]-data_x[index-1]
    dy = data_y[index]-data_y[index-1]
    dz = data_z[index]-data_z[index-1]
    
    step_vector = np.array([dx,dy,dz])   

    return step_vector  

def get_cos_theta(data_in):
    
    track_id_values = np.unique(data['TrackID'])
    cos_theta_avg = []
    
    for tid in track_id_values:
        print('track_id ={}'.format(tid))
        track_data = data[data['TrackID']==tid]
        x = track_data['Position X'].values
        y = track_data['Position Y'].values
        z = track_data['Position Z'].values
        
        length = len(track_data)
        #print(length)
        step_vectors = np.empty((length-1,3))
        cos_theta_temp = np.empty(length-2)
        for i in np.arange(length):
            if i>=1:
                
                step_vectors[i-1] = get_step_vector(i,x,y,z)
            if i>=2:
                
                v0 = step_vectors[i-2]
                v1 = step_vectors[i-1]
                cos_theta_temp[i-2] = np.dot(v0,v1)/(get_mod(v1)*get_mod(v0))

        if cos_theta_temp.size != 0:
            cos_theta_avg.append(np.nanmean(cos_theta_temp))
            print(cos_theta_temp)
        
    return cos_theta_avg

def dsample_get_cos_theta(data_in, ds_rate):
    """
    Args:
        data_in: pandas dataframe
        ds_rate: int, downsampling step size
    Returns:
        h_arr: 2D np array
    """

    for i in np.arange(ds_rate):
        
        downsampled_data = downsample(data_in, ds_rate, i)

        cos_theta_avgs = get_cos_theta(downsampled_data)
        #print(len(cos_theta_avgs))
            
        if i == 0:
            cos_theta_arr = cos_theta_avgs#,dtype=object)
        else:
            
            if len(cos_theta_avgs)>np.shape(cos_theta_arr)[0]:
                cos_theta_avgs=cos_theta_avgs[:-1]
            
            cos_theta_arr.append(cos_theta_avgs)
        
    return cos_theta_arr

def get_cos_theta_dict(filtered_data, step_sizes):
    """
    """
    cos_theta_dict = {}
    
    for i, s in enumerate(step_sizes):
        print('step_size ={}'.format(s))
        cos_theta_arr = dsample_get_cos_theta(filtered_data, s)
        cos_theta_dict["{}".format(s)] = cos_theta_arr
        
    return cos_theta_dict

def save_cos_theta_data(cos_theta_data, file_name):#, window, step_sizes, restriction):
    """
    """
    #with open('cos_theta_dict_'+'w{}_'.format(window)+'s{}_'.format(max(step_sizes))+'r{}_'.format(restriction)+file_name, 'w')
    with open('cos_theta_dict_'+file_name, 'w') as file:
        file.write(json.dumps(cos_theta_data))
      

In [58]:
filenames = np.array(['Control_frame001-200',
                      'Control_frame200-400',
                      'Control_frame400-600',
                      'Control_frame600-800',
                      'Control_frame800-1000',
                      'Control_frame1000-1200',
                      'LanB1_frame001-200',
                      'LanB1_frame200-400',
                      'LanB1_frame400-600',
                      'LanB1_frame600-800',
                      'LanB1_frame800-1000',
                      'LanB1_frame1000-1200',
                      'defLanB1_300817_frame200-400',
                      'defLanB1_300817_frame400-600'])

In [59]:
step_sizes = [1,2,3,4,5,6,7,8]
restriction = 5
window = 20

# iterate over data files and save H dictionary 
for i, file in enumerate(filenames[6:8]):
    print('opened file {}'.format(file))
    data = pd.read_csv('haemocyte_tracking_data/' + file + '.csv')
    filtered_data = filter_data_new(data,restriction)
    cos_theta_dict = get_cos_theta_dict(filtered_data, step_sizes)
    #print(cos_theta_dict)
    save_cos_theta_data(cos_theta_dict, file)

opened file LanB1_frame001-200
       Position X  Position Y  Position Z Unit  Time     TrackID      ID  \
0        121.4970     78.6212     10.6246   um     1  1000000000       0   
1        138.9500    153.3250     10.7094   um     1  1000000001       1   
2         67.7117    259.5530      0.5000   um     1  1000000002       2   
3         64.3338    304.3020      0.5000   um     1  1000000003       3   
4        152.6940    330.2540      0.5000   um     1  1000000004       4   
...           ...         ...         ...  ...   ...         ...     ...   
88585     96.3207    409.5130     70.5000   um   200  1000167233  175630   
88586     77.0704    414.9130     70.5000   um   200  1000175195  175631   
88587     96.6054    422.3520     70.5000   um   200  1000165370  175632   
88588    116.9660    426.8750     70.5000   um   200  1000166302  175633   
88589    112.2810    434.8530     70.5000   um   200  1000169602  175634   

       Absolute Time Unit.1  
0                0.0      

[-0.99475464 -0.92167941  0.99919421  0.82178796  0.91664798  0.25913092
  0.04274055 -0.11077601 -0.4349073   0.97078047 -0.57334068 -0.99986976
  0.23602702  0.04258278  0.18310408  0.88879597  0.92359577  0.08809554
  0.23038096 -0.97504629 -0.72504647  0.58392862  0.98385752 -0.1274313 ]
track_id =1000000017
[ 0.76597582  0.99943102  0.91555467  0.98980888  0.99911427  0.87445856
 -0.89184011  0.61192854 -0.85496813 -0.19682223  0.06492941  0.67323897
 -0.48694579 -0.999763    0.44516449 -0.50561316 -0.35201524 -0.06771244
 -0.85686353 -0.75600051  0.99123627 -0.99949871 -0.83222833  0.99202111
 -0.92360123 -0.66821962  0.08000658  0.73840348 -0.4891287   0.81390589
  0.33837524  0.24096049  0.86964052  0.58730194  0.87523576  0.4503322
  0.83306041  0.87595525  0.78717153 -0.99320158  0.8193093  -0.3760154
  0.18138645  0.5191082  -0.42357293 -0.96697801 -0.10027882  0.86170749
  0.81020063 -0.30586318 -0.47522874  0.95046877 -0.99996056  0.69696197
 -0.85940402 -0.48634991  0.875

track_id =1000000046
[ 0.66643561  0.7456434  -0.62521736  0.18588415]
track_id =1000000047
[-0.40118838  0.9869352   0.16138914 -0.9331281   0.45571674  0.95594079
  0.43048313  0.55431769 -0.67990083  0.98140852  0.60231138 -0.18657629
  0.25565286  0.29228297  0.68267906 -0.81531043 -0.92485936  0.33090102
  0.46041468  0.9630045  -0.16159167  0.86297439  0.74442566  0.3453634
 -0.3416791   0.95009671 -0.43883489  0.20601227  0.57766162  0.97905744
 -0.60240924 -0.21686582  0.06447692  0.31819657 -0.43602158  0.733449
  0.53585492  0.52774121  0.77653236 -0.81139545 -0.66810828 -0.92728519
  0.98923632  0.41701885  0.62105134  0.13579472  0.73790442 -0.43269388
  0.18625975  0.23904     0.83877768  0.72512877  0.508676    0.4303077 ]
track_id =1000000048
[ 0.60050383  0.27074077  0.34587316  0.65887535  0.97881552  0.4742108
  0.28552205 -0.70539342 -0.84275554  0.23646719 -0.16358963 -0.29959376
  0.95359851  0.65100845 -0.14875921  0.97727237 -0.10997983  0.33888782
  0.4164832   

[ 0.76180028 -0.54329801 -0.96442888 -0.92645619]
track_id =1000000068
[-0.93710622  0.34334765  0.16296455  0.84014101  0.9040525   0.70852747
  0.83053491  0.12128842  0.04061322 -0.81969652  0.14343213]
track_id =1000000069
[ 0.08317656 -0.83928456]
track_id =1000000070
[ 0.36226797 -0.39267784  0.69302151 -0.60802575 -0.93541753  0.10388684
  0.2109955   0.64468685  0.35801453  0.29062708 -0.62590475  0.32628164
  0.40245201  0.54917454 -0.86715106 -0.34619607  0.25197407 -0.97449099
  0.90662099]
track_id =1000000071
[ 0.371641   -0.21721299  0.85249756  0.52347406  0.85947304  0.51908385
  0.72160779  0.99299347  0.81088352]
track_id =1000000072
[0.6046485  0.76034869]
track_id =1000000073
[ 0.08616984  0.91828912  0.12672284  0.33999162 -0.82925356  0.29150013]
track_id =1000000074
[ 0.34216449  0.96626074 -0.18907086  0.45476022 -0.59409008]
track_id =1000000075
[ 0.94511122  0.36345279  0.89490695  0.67790057  0.59544777  0.93699143
 -0.90822016 -0.5181927  -0.39472946 -0.7156

[-0.79557173  0.68328305 -0.02141043  0.26933156  0.9669807  -0.12709973
  0.60602028  0.25528793  0.82619794  0.12140755  0.90551305  0.96153012
  0.04769819  0.29477124  0.66745442  0.89276558  0.81419567  0.99610818
 -0.52243502  0.87790727 -0.73373257 -0.66116177  0.05276895  0.75772035
  0.43397057 -0.07770903  0.93438701  0.56542887  0.93995556 -0.03658301
  0.74716394  0.66551357 -0.08844983  0.10413819  0.90903013  0.9767243
  0.85106842  0.99238248 -0.41236282 -0.48724376  0.7078204   0.69897972
  0.87079567 -0.41003143 -0.40437544  0.98862844  0.58507395  0.43893052
  0.84871198 -0.40543911  0.5506904  -0.62594486  0.26975953 -0.3412842
  0.72671976 -0.0212213   0.6203025   0.74746601  0.96600912 -0.17483235
  0.8125738  -0.30918009  0.99365945 -0.75965113 -0.89876938 -0.5830819
  0.68870788 -0.89222791 -0.23258728  0.5638393  -0.43088481 -0.05034152
  0.62783941  0.77546223 -0.24280081  0.86858201  0.13907875 -0.36909496
 -0.85970558 -0.0666759   0.53045476 -0.91270623 -0.47

...

track_id =1000145040
[-0.79926067 -0.46872543  0.68503941 -0.82882478]
track_id =1000145070
[ 0.16216621  0.90793982  0.66026657 -0.01827765 -0.27735743  0.32925611
 -0.82877902  0.75850526  0.93660207  0.60794385 -0.9286308 ]
track_id =1000145105
[0.95712424 0.86201336 0.93884831 0.92380084 0.82812702 0.95128577
 0.94611092 0.74884414 0.84018482 0.96128138 0.92017136 0.95018612
 0.50709296 0.98066018 0.50331046 0.92753598 0.63539812 0.80804096
 0.94596863 0.95095246 0.93164942]
track_id =1000145107
[0.99575764 0.98717568 0.9207095  0.54729273 0.67143712 0.92609867
 0.98316457 0.70075056 0.92645569 0.97781622 0.16983961 0.3415524
 0.54598825 0.76030801 0.23076869 0.76412458 0.94910751 0.90965171
 0.71275038 0.13869094]
track_id =1000145122
[ 0.59672425 -0.87443665  0.19648912  0.96884478 -0.16635131 -0.70260894
 -0.19608417  0.51351724 -0.58963765 -0.7476436   0.89762731  0.96787808
  0.53102925  0.25012222  0.14428711  0.66639579]
track_id =1000145132
[-0.99998706  0.98144829 -0.40680

[0.91584025 0.96997325 0.45108222 0.85378171 0.92376202 0.92808636
 0.99867668 0.93113639 0.93180721]
track_id =1000146530
track_id =1000146537
[ 0.03638492 -0.95060188  0.98942504  0.97048023  0.56668037  0.45228593
  0.71841368  0.94965515]
track_id =1000146544
track_id =1000146590
[0.72997229 0.76039078 0.99072844 0.50104639 0.6561296 ]
track_id =1000146602
[-0.56050165]
track_id =1000146608
[ 0.96367927  0.94590691  0.90959004  0.5481864  -0.16503972  0.91634841
  0.35754106 -0.87704459  0.93431725  0.95048925 -0.01300297  0.45352156
  0.89392913  0.56581902  0.94786074  0.16965443 -0.42283572  0.98033589
  0.96450658  0.98708154  0.83644774 -0.10039704  0.25008043  0.98320394
  0.99922118  0.08071731  0.79552537  0.29571797  0.42046251  0.99964796
  0.89593719  0.42514936 -0.2277236   0.98159381  0.93943956  0.96190003
  0.92084495  0.99079175 -0.05883065  0.1066317   0.67806397]
track_id =1000146634
[ 0.74975541  0.63088297  0.84856364 -0.10979032]
track_id =1000146638
[-0.284080

[ 0.57971466 -0.26465643  0.7367093   0.7465408 ]
track_id =1000147725
[-0.34113383  0.26220151  0.78053342 -0.49762078  0.21210132  0.63981582
  0.59251073  0.85680713  0.82798533  0.35065164  0.31680054]
track_id =1000147731
[-0.51302159 -0.96057238 -0.07971828 -0.59693075 -0.46551563  0.65305882
  0.12814913  0.79272635 -0.71098396]
track_id =1000147775
[ 0.67908596 -0.02001057  0.78301376  0.87999231  0.88809135  0.85285909
  0.96685206  0.62185978 -0.40550161 -0.01298093  0.87186079 -0.36160679
  0.96759201  0.63255359  0.17443282  0.98171611 -0.39215906 -0.06403199
 -0.6676379  -0.16063571 -0.76986922  0.64395127  0.63733503  0.6777576
  0.75573837  0.28785198 -0.37130641  0.9776663 ]
track_id =1000147778
[ 0.95434945 -0.23100728 -0.27131038 -0.55232403  0.10832281  0.37926121
  0.95217856 -0.68284595 -0.19081298 -0.95282859  0.76182939 -0.44279443]
track_id =1000147804
[0.77814419 0.49905879 0.9518914 ]
track_id =1000147813
[-0.34756183  0.37884415  0.97737356 -0.14125791 -0.448

[ 0.86975392 -0.50467997  0.0615288 ]
track_id =1000148822
[ 0.3936435   0.82047164  0.9983647   0.08192159 -0.18834975  0.99996691
  0.64060848  0.92418971  0.99999743  0.57925617 -0.25946222 -0.05190184
 -0.29156955  0.86559985 -0.95765185 -0.48312745  0.86280143]
track_id =1000148847
[0.9999867  0.88617701 0.96201752 0.95211363 0.9913711  0.93963975
 0.9735428 ]
track_id =1000148859
[0.92767078]
track_id =1000148863
[ 0.70274049  0.87754585  0.24260115 -0.83377466 -0.23408073]
track_id =1000148870
[0.99836059]
track_id =1000148880
[ 0.02591014  0.61234845  0.89810629  0.40944405  0.16014798 -0.8722224
 -0.95674527  0.27351257 -0.42548434  0.28371462  0.61421757  0.99719221
  0.70950627  0.98966261 -0.99516779  0.93331172  0.90229217  0.24565018
 -0.1301482   0.82752102 -0.08770792 -0.8172455   0.94663685 -0.71313901
  0.38297966 -0.08155463  0.95129375  0.77042824  0.9130763   0.92743189
  0.79008697]
track_id =1000148891
[-0.78548147  0.94942113  0.86390099  0.12025998 -0.4050371  

KeyboardInterrupt: 